#  ROHF comparison
In this notebook, we will use psi4 to generate some data. Then we will attempt to use an equivalent method to the ```CIS.py``` file in order to generate data for ROHF references. This will serve as a way to verify the code already written and as a catalyst for outline inspiration.

In [1]:
import psi4
import numpy as np

In [43]:
psi4.set_options({"basis":"sto-3g", "reference": "cuhf", "scf_type":"pk"})
allyl = psi4.geometry("""
0 2
H
C 1 r2
C 2 r3 1 a3
C 2 r3 1 a3 3 180.
H 3 r5 2 a5 1 0.
H 4 r5 2 a5 1 0.
H 3 r7 2 a7 1 180.
H 4 r7 2 a7 1 180.

r2=1.08424658
r3=1.40526604
r5=1.08095381
r7=1.08131649
a3=117.99450641
a5=121.41544408
a7=121.21891262
symmetry c1

""")
E, wfn = psi4.energy("scf", return_wfn = True)
E

-115.00893006028255

We will now copy the code from the ```CIS.py``` file and generate a datafile for ROHF. It can then be compared to the data in the CUHF files that are already generated.

In [44]:
def displayCISHamiltonian(wfn, geom):
    """displays the CIS hamiltonian in MO basis"""
    # getting the orbital energies
    
    epsilon_a, C_a = wfn.epsilon_a().np, wfn.Ca().np
    epsilon_b, C_b = wfn.epsilon_b().np, wfn.Cb().np
    epsilon = np.append(epsilon_a, epsilon_b) # accounts for the fact that the energies might be different
    sortedorder = np.argsort(epsilon)
    epsilon = np.sort(epsilon)
    
    # make the C matrix => it contains all the orbitals, both alpha and beta
    C = np.block([[C_a, np.zeros(C_a.shape)], [np.zeros(C_b.shape), C_b]]) # accounts for the fact that the orbitals might be different (uhf, cuhf)
    C = C[:, sortedorder] # sort the eigenfunctions
    
    basis = wfn.basisset()
    mints = psi4.core.MintsHelper(basis)

    # getting the two electron integrals in correct basis => we need it in MO basis
    tei = mints.ao_eri().np # given in chemists notation

    #change the basis of the tei
    tei_int = np.kron(np.eye(2), tei)
    tei_big = np.kron(np.eye(2), tei_int.T)

    tei_ao = tei_big.transpose(0, 2, 1, 3) - tei_big.transpose(0, 2, 3, 1) # accounts for both coulomb and exchange, switch to physisists notation
    tei_mo = np.einsum("pQRS,pP->PQRS", np.einsum("pqRS,qQ->pQRS", np.einsum("pqrS,rR->pqRS", np.einsum("pqrs,sS->pqrS", tei_ao, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)
    
    occupied = wfn.nalpha() + wfn.nbeta()
    available = mints.nbf()*2
    virtual = available - occupied
    #getting the excitations
    excitations = []
    for orbital in range(occupied): # for every occupied orbital
        for another_orbital in range(occupied, available): # we can make an excitation to every virtual orbital
            excitations.append((orbital, another_orbital))
    # getting the hamiltonian
    dim = occupied*virtual
    H_cis = np.zeros((dim, dim))
    for row, excitation in enumerate(excitations):
        i, a = excitation
        for collumn, another_excitation in enumerate(excitations):
            j, b = another_excitation   
            H_cis[row, collumn] = (epsilon[a] - epsilon[i])*(i == j)*(a == b) + tei_mo[a, j, i, b]
    

    # get the E_0 value in the hamiltonian
    extra_row = np.zeros((H_cis.shape[1],))
    H_cis = np.vstack((extra_row, H_cis))
    extra_collumn = np.zeros((H_cis.shape[0], 1))
    H_cis = np.hstack((extra_collumn, H_cis))
    H_cis[0,0] = E
    return H_cis, excitations
ham, excitations = displayCISHamiltonian(wfn, allyl)

In [45]:
from scipy.linalg import eigh
exits, coefs = eigh(ham)

In [46]:
def GetExitations(exits, coefs, excitations, filepath="NoNameGiven"):
        """Get the excitation energies and the contributions"""
        if filepath == "NoNameGiven":
            raise ValueError("no path specified")
        from pathlib import Path
        Path(f"{filepath}").touch()
        datafile = open(f"{filepath}", "w")
        contrib = coefs[1:, 1:]**2
        energies = exits[1:]
        counterdict = {} # added to check how many times a certain excitation occurs
        datafile.writelines(f"scf energy for ROHF: {E}\n")
        for state, energy in enumerate(energies):
            datafile.writelines(f" {state + 1} : {energy}\n")
            for excitation, contribution in enumerate(contrib[:, state]):
                if contribution*100 > 1:
                    datafile.writelines(f"\t{contribution:.3f} : {excitations[excitation]}\n")
                    if excitations[excitation] not in counterdict:
                        counterdict[excitations[excitation]] = 0
                    counterdict[excitations[excitation]] += 1   
        datafile.close()
GetExitations(exits, coefs, excitations, "/user/gent/440/vsc44013/Bachelor_project/Bachelor_project/definitive_versions/notebooks/ROHF_CUHF/CUHF_cis.txt")